In [4]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image
import os
import numpy as np 

In [5]:
# initializing mtcnn for face detection
mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) 
# initializing resnet for face img to embeding conversion using pretrained model VGGface2
resnet = InceptionResnetV1(pretrained='vggface2').eval() 

# Prompt to use gpu if present 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = resnet.to(device)

In [8]:
# Training dataset path(edit here)
dataset=datasets.ImageFolder(r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Train") 

idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

idx_to_class

{0: 'George Clooney',
 1: 'Hugh Jackman',
 2: 'Joaquin Phoenix',
 3: 'Kate Winslet',
 4: 'Leonardo DiCaprio',
 5: 'Matt Damon',
 6: 'Miley Cyrus',
 7: 'Morgan Freeman',
 8: 'Shakira',
 9: 'Will Smith'}

In [7]:
def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        face = face.to("cuda")
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list

In [9]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file


#### The function *face_match()* reads the image from the *img_path* passed to it and crops the face in the picture, done by mtcnn(), then it stores the processed face image into an embedded matrix which is then iterated to find a minimum euclidean distance the image corresponding to the matrix having least euclidean distance.



In [10]:

def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    face = face.to("cuda")
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))

In [11]:
#call the function to find the closest match to the embedded matrix which is created when passing the image path
result = face_match(r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Test\Shakira\test__000073-000044.jpg", 'data.pt')

print('Face matched with: ',result[0], 'With distance: ',result[1])

Face matched with:  Morgan Freeman With distance:  0.5252143144607544


#### The code below will iterate all the images in the folder path given and predict which class(person) the image belongs to

In [16]:
# Set the path to the folder containing the images(Edit Here)
folder_path = r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Test\Leonardo DiCaprio"


for filename in os.listdir(folder_path):
        
    image_path = os.path.join(folder_path, filename)
    result = face_match(image_path, 'data.pt')

    print('\nFace matched with: ',result[0], 'with distance: ',result[1])


Face matched with:  Leonardo DiCaprio with distance:  0.4954832196235657

Face matched with:  Leonardo DiCaprio with distance:  0.7011120319366455

Face matched with:  Leonardo DiCaprio with distance:  0.5445054173469543

Face matched with:  Leonardo DiCaprio with distance:  0.48734161257743835

Face matched with:  Matt Damon with distance:  0.8133037686347961

Face matched with:  Leonardo DiCaprio with distance:  0.5050778985023499

Face matched with:  Leonardo DiCaprio with distance:  0.4576975107192993

Face matched with:  Leonardo DiCaprio with distance:  0.4508529007434845

Face matched with:  Leonardo DiCaprio with distance:  0.6753764152526855

Face matched with:  Leonardo DiCaprio with distance:  0.6856589913368225

Face matched with:  Leonardo DiCaprio with distance:  0.7874928712844849

Face matched with:  Leonardo DiCaprio with distance:  0.43504637479782104

Face matched with:  Leonardo DiCaprio with distance:  0.6974965333938599

Face matched with:  Leonardo DiCaprio with